## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from configvs import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database1.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,54.79,64,91,16.71,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,92.28,74,75,11.50,broken clouds
2,2,Caala,AO,-12.8525,15.5606,55.11,36,15,2.64,few clouds
3,3,Cockburn Town,TC,21.4612,-71.1419,82.36,74,40,17.27,scattered clouds
4,4,Bubaque,GW,11.2833,-15.8333,80.55,83,87,11.95,light rain


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Cockburn Town,TC,21.4612,-71.1419,82.36,74,40,17.27,scattered clouds
4,4,Bubaque,GW,11.2833,-15.8333,80.55,83,87,11.95,light rain
7,7,Matara,LK,5.9485,80.5353,80.19,82,99,15.21,overcast clouds
8,8,Alofi,NU,-19.0595,-169.9187,77.07,76,96,23.80,overcast clouds
11,11,Port Blair,IN,11.6667,92.7500,81.50,83,100,20.09,heavy intensity rain
15,15,Bambang,PH,16.3833,121.1000,75.90,92,76,0.47,broken clouds
16,16,Salalah,OM,17.0151,54.0924,77.09,100,100,9.22,light intensity drizzle
20,20,Vaini,TO,-21.2000,-175.2000,78.96,73,40,12.66,scattered clouds
25,25,Biak,ID,-0.9131,122.8766,75.54,89,64,5.08,broken clouds
27,27,Qunduz,AF,36.7290,68.8570,80.71,15,0,3.11,clear sky


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# There are no empty rows

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cockburn Town,TC,82.36,scattered clouds,21.4612,-71.1419,
4,Bubaque,GW,80.55,light rain,11.2833,-15.8333,
7,Matara,LK,80.19,overcast clouds,5.9485,80.5353,
8,Alofi,NU,77.07,overcast clouds,-19.0595,-169.9187,
11,Port Blair,IN,81.50,heavy intensity rain,11.6667,92.7500,
15,Bambang,PH,75.90,broken clouds,16.3833,121.1000,
16,Salalah,OM,77.09,light intensity drizzle,17.0151,54.0924,
20,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,
25,Biak,ID,75.54,broken clouds,-0.9131,122.8766,
27,Qunduz,AF,80.71,clear sky,36.7290,68.8570,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cockburn Town,TC,82.36,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
4,Bubaque,GW,80.55,light rain,11.2833,-15.8333,"Casa Dora, Bubaque"
7,Matara,LK,80.19,overcast clouds,5.9485,80.5353,Amaloh Boutique Resort
8,Alofi,NU,77.07,overcast clouds,-19.0595,-169.9187,Taloa Heights
11,Port Blair,IN,81.50,heavy intensity rain,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
15,Bambang,PH,75.90,broken clouds,16.3833,121.1000,WaterBlu Resort
16,Salalah,OM,77.09,light intensity drizzle,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
20,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
25,Biak,ID,75.54,broken clouds,-0.9131,122.8766,Penginapan Kim
27,Qunduz,AF,80.71,clear sky,36.7290,68.8570,Spin Zar cotton company


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))